<a href="https://colab.research.google.com/github/ttcielott/Deep-Learning-with-PyTorch/blob/main/Chapter4_Under_the_hood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning for Coders with Fastai and PyTorch
## Chapter 4. Under the Hood: Training a Digit Classifier


In [ ]:
3